In [ ]:
import pandas as pd, numpy as np

In [ ]:
train = pd.read_csv(r'C:\Users\Anjana Tiha\Drive D\Programming\data\house-prices-advanced-regression-techniques\train.csv')
test = pd.read_csv(r'C:\Users\Anjana Tiha\Drive D\Programming\data\house-prices-advanced-regression-techniques\train.csv')

In [3]:
data = train+test

In [4]:
target_col = 'SalePrice'

target = data[target_col]

data.drop(columns=target_col, inplace=True)

features = data

rating_cols = ['ExterQual' 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
rating_score = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0}
rating_cols2 = {'BsmtFinType1', 'BsmtFinType2'}
rating_score2 = {'GLQ':6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0}

ignore_cols = ['BsmtExposure']
features.drop(columns=ignore_cols, inplace=True)

features.describe()
# target.describe()


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,1461.000000,113.794521,140.099917,21033.656164,12.198630,11.150685,3942.535616,3969.731507,207.370523,887.279452,...,945.960274,188.489041,93.320548,43.908219,6.819178,30.121918,5.517808,86.978082,12.643836,4015.631507
std,843.220019,84.601142,48.569504,19962.529865,2.765993,2.225599,60.405808,41.290814,362.132413,912.196182,...,427.609683,250.677589,132.512055,122.238297,58.634661,111.514831,80.354614,992.246049,5.407252,2.656190
min,2.000000,40.000000,42.000000,2600.000000,2.000000,2.000000,3744.000000,3900.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,4012.000000
25%,731.500000,40.000000,118.000000,15107.000000,10.000000,10.000000,3908.000000,3934.000000,0.000000,0.000000,...,669.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,4014.000000
50%,1461.000000,100.000000,138.000000,18957.000000,12.000000,10.000000,3946.000000,3988.000000,0.000000,767.000000,...,960.000000,0.000000,50.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,4016.000000
75%,2190.500000,140.000000,160.000000,23203.000000,14.000000,12.000000,4000.000000,4008.000000,332.000000,1424.500000,...,1152.000000,336.000000,136.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16.000000,4018.000000
max,2920.000000,380.000000,626.000000,430490.000000,20.000000,18.000000,4020.000000,4020.000000,3200.000000,11288.000000,...,2836.000000,1714.000000,1094.000000,1104.000000,1016.000000,960.000000,1476.000000,31000.000000,24.000000,4020.000000


In [5]:
from sklearn import preprocessing

# print(data.isnull().sum(axis=1))   

features.replace(-np.Inf, np.nan)
features.replace(np.Inf, np.nan)


dtype_grp = features.columns.to_series().groupby(data.dtypes).groups

for dtype in dtype_grp:
    print(dtype_grp[dtype])

    
for dtype in dtype_grp:
    if dtype=='int64' :
        for col in dtype_grp[dtype]:
            features[col] = features[col].fillna(0.0)
            features[col]=preprocessing.normalize([features[col]], norm='l2').flatten()
            features[col] = features[col].astype('float64')
    elif dtype=='float64' :
        for col in dtype_grp[dtype]:
            features[col] = features[col].fillna(0.0)
            features[col]=preprocessing.normalize([features[col]], norm='l2').flatten()
            features[col] = features[col].astype('float64')
    elif dtype=='object' :
        for col in dtype_grp[dtype]:
            if col in rating_cols:
                features[col].replace(rating_score, regex=True, inplace=True)
                features[col] = features[col].fillna(0.0)
                features[col] = features[col].astype('float64')
            elif col in rating_cols2:
                features[col].replace(rating_score2, regex=True, inplace=True)
                features[col] = features[col].fillna(0.0)
                features[col] = features[col].astype('float64')
            else:
                try:
                    le = preprocessing.LabelEncoder()
                    enc = preprocessing.OneHotEncoder()
                    le.fit(features[col].astype(str))
                    label_enc_feature_val = le.transform(features[col].astype(str))
                    features[col] = label_enc_feature_val.astype('float64')

#                     enc.fit([label_enc_feature_val])  
#                     hot_enc_feature_val = enc.transform([label_enc_feature_val])

#                     print(hot_enc_feature_val)

#                     features.join(hot_enc_feature_val)
#                     features.drop(columns=col)
                except ValueError:
                    print(col)
                    print("Oops!  That was no valid number.  Try again...")



Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')
Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object')
Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 

In [6]:
X = features.values
Y = target
# print(features.info())

In [7]:
from sklearn.model_selection import KFold

from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score
    
def model_evaluation(X, Y, splitter, model, report, details):
    explained_variance_score_val = 0
    mean_absolute_error_val = 0
    mean_squared_error_val = 0
    mean_squared_log_error_val = 0
    median_absolute_error_val = 0
    r2_score_val = 0
    i=0
    if report:
        print("*"*50, " START ", "*"*50)
#         print("Spliter Description:")
#         print(splitter)
#         print("-"*100, "\n")
        print("Model Description:")
        print(model)
        print("-"*100,"\n")
    
    for train_index, test_index in splitter.split(X, Y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        
        # model fitting
        model.fit(X_train, y_train)
        
        # prediction
        predict = model.predict(X_test)

        # evaluation scores
        explained_variance_score_temp = explained_variance_score(y_test, predict)
        mean_absolute_error_temp = mean_absolute_error(y_test, predict)
        mean_squared_error_temp = mean_squared_error(y_test, predict)
        mean_squared_log_error_temp = mean_squared_log_error(y_test, predict)
        median_absolute_error_temp = median_absolute_error(y_test, predict)
        r2_score_temp = r2_score(y_test, predict)
        
        explained_variance_score_val = explained_variance_score_val + explained_variance_score_temp
        mean_absolute_error_val = mean_absolute_error_val + mean_absolute_error_temp
        mean_squared_error_val = mean_squared_error_val + mean_squared_error_temp
        mean_squared_log_error_val = mean_squared_log_error_val + mean_squared_log_error_temp
        median_absolute_error_val = median_absolute_error_val + median_absolute_error_temp
        r2_score_val = r2_score_val + r2_score_temp
        
        if details:
            print("*"*25,  " ITERATION - ", i+1, "*"*25)
            #print("TRAIN:", train_index, "TEST:", test_index)
            print("explained_variance_score_temp: ", explained_variance_score_temp)
            print("mean_absolute_error_temp: ", mean_absolute_error_temp)
            print("mean_squared_error_temp: ", mean_squared_error_temp)
            print("mean_squared_log_error_temp: ", mean_squared_log_error_temp)
            print("median_absolute_error_temp: ", median_absolute_error_temp)
            print("r2_score_temp: ", r2_score_temp)
            
#             print("-"*35)
#             print(metrics.classification_report(y_test, predict))
#             print("-"*35)
#             print("confusion Matrix:\n\n", metrics.confusion_matrix(y_test, predict))
#             print("-"*35)
            print("\n")
        
        i+=1
    
    split_num = splitter.get_n_splits()
    
    explained_variance_score_val = explained_variance_score_val/split_num
    mean_absolute_error_val = mean_absolute_error_val/split_num
    mean_squared_error_val = mean_squared_error_val/split_num
    mean_squared_log_error_val = mean_squared_log_error_val/split_num
    median_absolute_error_val = median_absolute_error_val/split_num
    r2_score_val = r2_score_val/split_num
    
    if report:
        print("*"*50, " Average For", i+1, " Folds", "*"*50)
        print("\n")
        print('%50s%s' % ("Average explained_variance_score: ", explained_variance_score_val))
        print('%50s%s' % ("Average mean_absolute_error: ", mean_absolute_error_val))
        print('%50s%s' % ("Average mean_squared_error: ", mean_squared_error_val))
        print('%50s%s' % ("Average mean_squared_log_error: ", mean_squared_log_error_val))
        print('%50s%s' % ("Average median_absolute_error: ", median_absolute_error_val))
        print('%50s%s' % ("Average r2_score: ", r2_score_val))
        print("\n")
        print("*"*100)
#         print("*"*50, " END ", "*"*50)
    
    return explained_variance_score_val, mean_absolute_error_val, mean_squared_error_val, mean_squared_log_error_val, median_absolute_error_val, r2_score_val


In [8]:
from sklearn.model_selection import train_test_split

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import (AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomTreesEmbedding, RandomForestRegressor, VotingClassifier)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.isotonic import IsotonicRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ARDRegression, LinearRegression, LogisticRegression, LogisticRegressionCV, logistic_regression_path, HuberRegressor, PassiveAggressiveRegressor, RandomizedLogisticRegression, RANSACRegressor, SGDRegressor, TheilSenRegressor
from sklearn.mixture import GaussianMixture
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB 
from sklearn.neighbors import KDTree, KNeighborsRegressor, NearestNeighbors, RadiusNeighborsRegressor
from sklearn.neural_network import BernoulliRBM, MLPRegressor
from sklearn.svm import LinearSVR, NuSVR, SVR
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor



kf = KFold(n_splits = 5, random_state=None, shuffle =True)


regressors = {
    "AdaBoostRegressor": AdaBoostRegressor(),
    "ARDRegression": ARDRegression(),
    "BaggingRegressor": BaggingRegressor(),
#     "BernoulliRBM": BernoulliRBM(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "ExtraTreesRegressor": ExtraTreesRegressor(),
    "ExtraTreeRegressor": ExtraTreeRegressor(),
#     "GaussianMixture": GaussianMixture(),
#     "GaussianNB": GaussianNB(),
    "GaussianProcessRegressor": GaussianProcessRegressor(),
    "GradientBoostingRegressor": GradientBoostingRegressor(),
    "HuberRegressor": HuberRegressor(),
#     "IsotonicRegression": IsotonicRegression(),
    "KernelRidge": KernelRidge(),
#     "KDTree": KDTree(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "LinearRegression": LinearRegression(), 
    "LogisticRegression": LogisticRegression(),
    "LogisticRegressionCV": LogisticRegressionCV(),
#     "logistic_regression_path": logistic_regression_path(),
    "LinearSVR": LinearSVR(),
    "MLPRegressor": MLPRegressor(),
#     "MultinomialNB": MultinomialNB(),
    "NuSVR": NuSVR(),
    "PassiveAggressiveRegressor": PassiveAggressiveRegressor(),
#     "QuadraticDiscriminantAnalysis": QuadraticDiscriminantAnalysis(),
    "RadiusNeighborsRegressor": RadiusNeighborsRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "RandomizedLogisticRegression": RandomizedLogisticRegression(),
    "RANSACRegressor": RANSACRegressor(),
    "SGDRegressor": SGDRegressor(),
    "SVR": SVR(),
    "TheilSenRegressor": TheilSenRegressor(),
}
    
    
splitter = kf 
report = 1
details = 1


evaluation = {}

print("Spliter Description:")
print(splitter)
        
        
for name in regressors:
    evaluation_temp = []
    
    explained_variance_score_val, mean_absolute_error_val, mean_squared_error_val, mean_squared_log_error_val, median_absolute_error_val, r2_score_val = model_evaluation(X, Y, splitter, regressors[name], report, details=None)
    evaluation_temp.append(explained_variance_score_val)
    evaluation_temp.append(mean_absolute_error_val)
    evaluation_temp.append(mean_squared_error_val)
    evaluation_temp.append(mean_squared_log_error_val)
    evaluation_temp.append(median_absolute_error_val)
    evaluation_temp.append(r2_score_val)
    evaluation[name] = evaluation_temp
    

rows_list = []
for name in evaluation:
    rows_list.append([name]+evaluation[name])
                           
evaluation_pd = pd.DataFrame(rows_list, columns=['explained_variance_score',  'mean_absolute_error', 'mean_squared_error', 'mean_squared_log_error', 'median_absolute_error', 'r2_score']) 

C:\Users\Anjana Tiha\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class RandomizedLogisticRegression is deprecated; The class RandomizedLogisticRegression is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


Spliter Description:
KFold(n_splits=5, random_state=None, shuffle=True)
**************************************************  START  **************************************************
Model Description:
AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None)
---------------------------------------------------------------------------------------------------- 

**************************************************  Average For 6  Folds **************************************************


                Average explained_variance_score: 0.8265867530935708
                     Average mean_absolute_error: 47180.4798457463
                      Average mean_squared_error: 4578030696.787595
                  Average mean_squared_log_error: 0.03874169580550478
                   Average median_absolute_error: 34447.30688665347
                                Average r2_score: 0.8171945368643984


***************************************

**************************************************  Average For 6  Folds **************************************************


                Average explained_variance_score: 0.8795754050198991
                     Average mean_absolute_error: 32935.86127838013
                      Average mean_squared_error: 3083002267.1558437
                  Average mean_squared_log_error: 0.01825638168594178
                   Average median_absolute_error: 21822.646845697254
                                Average r2_score: 0.8788637082797985


****************************************************************************************************
**************************************************  START  **************************************************
Model Description:
HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=100,
        tol=1e-05, warm_start=False)
---------------------------------------------------------------------------------------------------- 

***********

C:\Users\Anjana Tiha\Anaconda3\lib\site-packages\sklearn\metrics\regression.py:314: RuntimeWarning: invalid value encountered in log
  return mean_squared_error(np.log(y_true + 1), np.log(y_pred + 1),


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import figure


figure(num=None, figsize=(14, 6), dpi=250)

labels= ['accuracy', 'precision', 'recall', 'f1']
ax = plt.subplot(111)

for n in range(0,4):
    plt.plot([name for name in evaluation],[evaluation[name][n] for name in evaluation], label = labels[n])

leg = plt.legend(loc='best', ncol=2, mode="expand", shadow=True, fancybox=True)
plt.xticks(rotation=45)
# leg.get_frame().set_alpha(0.5)
plt.legend()
ax.tick_params(labelsize='large', width=5)
ax.grid(True, linestyle='-.')

plt.tight_layout()
plt.xlabel('x label')
plt.ylabel('y label')

plt.title("TITLE")
plt.show()
